In [1]:
from pathlib import Path
from sim import rebforces
import numpy as np

gas_profile_root = Path("disk/calculated_profiles/")
gas_profile_name = "20220721"
gas_profile = np.load(gas_profile_root / gas_profile_name / "all_variables.npz")

rebforces.set_profiles(
    len(gas_profile["r"]),
    gas_profile["r"],
    gas_profile["velocity"].T * rebforces.CM_PER_S,
    gas_profile["rho_0"] * rebforces.G_PER_CM3,
    1.0
)

rebforces.copy_np_to_c(gas_profile["sigma"] * rebforces.G_PER_CM3 / 14959787070000, rebforces.SURFACE_DENSITY_PROF, rebforces.STD_PROF_N.value)
rebforces.copy_np_to_c(gas_profile["H"] / gas_profile["r_cm"], rebforces.SCALE_HEIGHT_PROF, rebforces.STD_PROF_N.value)
rebforces.copy_np_to_c(gas_profile["torque"], rebforces.TORQUE_PROF, rebforces.STD_PROF_N.value)

In [10]:
import rebound


def measure_one_orbit_torque(m=1.0, force=None, a=0.23, e=0.02, i=0.01):
    sim = rebound.Simulation()
    sim.G = 4 * np.pi ** 2

    primary = rebound.Particle(m=1.)
    sim.add(primary)

    rho_p = 3.0 * rebforces.G_PER_CM3.value
    r = (3e-6 * 3 / (rho_p * 4 * np.pi))**(1./3)
    sim.add(rebound.Particle(simulation=sim, primary=primary, m=m * 3e-6, r=r, a=a, e=e, inc=i))
    sim.move_to_com()

    sim.integrator = 'IAS15'

    sim.additional_forces = force
    sim.force_is_velocity_dependent = 1

    h_i = sim.particles[1].h
    dt = sim.particles[1].P * 2
    sim.integrate(dt)
    h_f = sim.particles[1].h

    return m * 3e-6 * (h_f - h_i) / dt

for m in (0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0):
    for a in (0.20, 0.21, 0.22, 0.23, 0.24, 0.25):
        for e in (0.0, 0.01, 0.05):
            for i in (0.0, 0.01, 0.05):
                drag = measure_one_orbit_torque(m, rebforces.IOPF_drag_all, a, e, i)
                migration = measure_one_orbit_torque(m, rebforces.IOPF_torque_all, a, e, i)
                ratio = migration / drag

                print(f"{m=} {a=} {e=} {i=} {drag=} {migration=} {ratio=:.3e} STRONGER={'MIGRATION' if abs(migration) > abs(drag) else 'DRAG'}")

m=0.01 a=0.2 e=0.0 i=0.0 drag=5.559952140605185e-15 migration=1.6461978149156079e-15 ratio=2.961e-01 STRONGER=DRAG
m=0.01 a=0.2 e=0.0 i=0.01 drag=5.757915151608462e-15 migration=1.6461977404395821e-15 ratio=2.859e-01 STRONGER=DRAG
m=0.01 a=0.2 e=0.0 i=0.05 drag=1.2254154267744231e-14 migration=1.6461979638676592e-15 ratio=1.343e-01 STRONGER=DRAG
m=0.01 a=0.2 e=0.01 i=0.0 drag=2.4918380980675395e-14 migration=1.6461978149156079e-15 ratio=6.606e-02 STRONGER=DRAG
m=0.01 a=0.2 e=0.01 i=0.01 drag=2.5742807646719645e-14 migration=1.646197889391634e-15 ratio=6.395e-02 STRONGER=DRAG
m=0.01 a=0.2 e=0.01 i=0.05 drag=4.554844364658636e-14 migration=1.6461978893916324e-15 ratio=3.614e-02 STRONGER=DRAG
m=0.01 a=0.2 e=0.05 i=0.0 drag=2.1367798329415981e-13 migration=1.6461978149156079e-15 ratio=7.704e-03 STRONGER=DRAG
m=0.01 a=0.2 e=0.05 i=0.01 drag=2.1740419469855595e-13 migration=1.6461978149156085e-15 ratio=7.572e-03 STRONGER=DRAG
m=0.01 a=0.2 e=0.05 i=0.05 drag=3.06777689999588e-13 migration=1.6